# Purpose
- This file is a scratch-pad for sql queries

In [2]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

c:\Users\MatthewLyn\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
%sql referralsDB

In [4]:
%%sql
CREATE OR REPLACE TABLE coc as

SELECT DISTINCT * FROM read_csv_auto('../data/reference/coc.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
818829


In [5]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/user_list.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
266


In [6]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [7]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [8]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
86137


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [12]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

WITH main as (

    SELECT DISTINCT rf.*
    , COALESCE(tm.Team, 'N/A') as User_Team

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , c.health_plan
    , c.care_partner_name
    , c.care_partner_group

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team as tm
        on  "User_FName" = tm.Fname
        AND "User_LName" = tm.Lname

    LEFT JOIN coc as c
        ON rf."Plan ID" = c.member_id
        AND date_part('month', "Referral Date":: DATE) = c.month_ym:: INTEGER
        AND date_part('year', "Referral Date":: DATE) = c.year_ym:: INTEGER

    WHERE "Visit Status" IS NOT NULL
)

, max_updatedt as (
    SELECT DISTINCT
        Referral_keyid
        , Update_DT
        , MAX(Update_DT) OVER (PARTITION BY Referral_keyid) as Last_UpdateDT

    FROM main 
)

SELECT DISTINCT m.* 

FROM main as m

INNER JOIN max_updatedt as dt
    ON m.Update_DT = dt.Last_UpdateDT
    AND m.Referral_keyid = dt.Referral_keyid

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
83421


In [13]:
%%sql

SELECT DISTINCT * FROM bi_referrals

LIMIT 10

*  (referralsDB) duckdb:///../data/referral.db
Done.


C0,Center,Clinic,Referring Provider,Referring Provider NPI,Referral Date,Last Name,First Name,Sex,DOB,pat Status,Language,Address,Mobile Phone,Home Phone,PCP,Referred to Specialist,Specialist NPI,Referral Status,Network,Specialty,Diagnosis,Procedure,Approval Status,Visit Status,Referral Type,Patient Type,Health Plan,Plan ID,LOB,Auth No,Last Update,Update_DT,file_source,User_Fname,User_LName,User_Team,patient_name,health_plan,care_partner_name,care_partner_group,patient_id,Referral_keyid,Updated Status,Proc_code,Proc_name,Fmt Mobile Phone,Fmt Home Phone,Diagnosis Description
6963,"Centrum Medical Holdings, LLC",CH - MIRAMAR PARK,"Jose I Del Risco, APRN",1639773377,2023-03-16,WILLIAMS,CARLY,F,1900-12-06,Active,English,"6751 N UNIVERSITY DR APT 316TAMARAC, FL, 33321",7542141335,7542141335,"Del Risco, Jose","Dr. Monica M. Companioni, MD",None,Open,Out of network,Gynecology,Z12.4,OFFICE OUTPATIENT NEW 45 MINUTES - 99204,Approved (comments required),Scheduled visit,External,Adult,Ambetter (Centrum Contract),U9321914502,Exchange,REFDFFBE94C6,"DEPOSADA, YORDAY",Mar 21 2023 10:04AM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_29.xlsx,YORDAY,DEPOSADA,Referral Coordinator,WILLIAMSCARLY,AMBETTER FL IFP,None,None,WILLIAMSCARLY1900-12-06,WILLIAMSCARLY1900-12-06U932191450216397733772023-03-16OFFICE OUTPATIENT NEW 45 MINUTES - 99204Gynecology,Open,99204,OFFICE/OUTPATIENT VISIT NEW,(754) 421-4133,(754) 421-4133,ENCOUNTER OTHER SCREENING MALIG NEOPLASM CERVIX
769,"Centrum Medical Holdings, LLC",LMMC - Coral Way,"Alcides Morejon, ARNP",1194288951,2023-03-06,HERNANDEZ,FRANCIS,F,1967-10-08,Active,Spanish,"9971 SW 41ST STMiami, FL, 33165",7864867880,7864867880,"Morejon, Alcides",aPPT PENDING BY PATIENT,None,Open,Out of network,Diagnostic Radiology,Z12.31,"MAMMOGRAM, SCREENING - 77057",Reviewed- Ready to be processed,Pending,External,Adult,Oscar Insurance Company of Florida,OSC79497397-01,Exchange,N/A,"CARTAYA, IRENE",Mar 6 2023 8:16AM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_29.xlsx,IRENE,CARTAYA,N/A,HERNANDEZFRANCIS,OSCAR IFP,None,None,HERNANDEZFRANCIS1967-10-08,"HERNANDEZFRANCIS1967-10-08OSC79497397-0111942889512023-03-06MAMMOGRAM, SCREENING - 77057Diagnostic Radiology",Open,77057,None,(786) 648-6788,(786) 648-6788,ENCOUNTER SCREENING MAMMO MALIG NEOPLASM BREAST
797,"Centrum Medical Holdings, LLC",CH - FT. LAUDERDALE,"Decire Espinoza de Mendoza, APRN",1851983316,2023-04-05,CHAVARRIA,ALBERTO,M,1962-05-03,Active,English,"4121 Stirling Rd Apt 309Dania Beach, FL, 33314",9548597642,9548597642,"Espinoza de Mendoza, Decire",None,None,Open,Out of network,Urology,R10.814,None,HP Pending,Pending Appt,External,Adult,Ambetter (Centrum Contract),U9668211501,Exchange,N/A,"TORRES, NIURKA",Apr 5 2023 12:48PM,../data/raw_referrals/excels/FLORIDA REFERRALS 3_31-4_7.xlsx,NIURKA,TORRES,N/A,CHAVARRIAALBERTO,AMBETTER FL IFP,None,None,CHAVARRIAALBERTO1962-05-03,CHAVARRIAALBERTO1962-05-03U966821150118519833162023-04-05NAUrology,Open,None,None,(954) 485-9764,(954) 485-9764,LEFT LOWER QUADRANT ABDOMINAL TENDERNESS
6502,"Centrum Medical Holdings, LLC",LMMC - Eureka,"Ivan Rapado Viera, MD",1295026243,2023-03-08,ARTEAGA,BEATRIZ,F,1976-09-28,Active,Spanish,"13445 SW 251st LnHomestead, FL, 33032",3058019398,3058019398,"Rapado Viera, Ivan",None,None,Open,Out of network,Diagnostic Radiology,Z12.31,"MAMMOGRAM, SCREENING - 77057",Complete/no Auth# needed,Completed,External,Adult,Oscar Insurance Company of Florida,OSC79321726-01,Exchange,...,"GARCIA, YISEL",Mar 8 2023 10:50AM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_31.xlsx,YISEL,GARCIA,N/A,ARTEAGABEATRIZ,OSCAR IFP,None,None,ARTEAGABEATRIZ1976-09-28,"ARTEAGABEATRIZ1976-09-28OSC79321726-0112950262432023-03-08MAMMOGRAM, SCREENING - 77057Diagnostic Radiology",Open,77057,None,(305) 580-1939,(305) 580-1939,ENCOUNTER SCREENING MAMMO MALIG NEOPLASM BREAST
5722,"Centrum Medical Holdings, LLC",CH - E Hialeah,"Heliodoro Ruiz, MD",1831529932,2023-03-23,VELASCO,NELSON,M,1975-11-16,Active,Spanish,"650 E 21 ST 33013HIALEAH, FL, 